In [13]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData/inst/extdata/kim/eclipse34_debug.arff') 
df = pd.DataFrame(data[0])

In [14]:
df.head()

,LOC,AverageComplexity,MaximumComplexity,LCOM,DIT,IFANIN,CBO,NOC,RFC,NIM,NIV,NCM,NCV,WMC,change_times,plus,minus,defect_num,isDefective
0,588.0,2.0,7.0,97.0,2.0,1.0,54.0,0.0,65.0,20.0,11.0,23.0,23.0,43.0,166.0,2210.0,3179.0,8.0,b'TRUE'
1,21.0,1.0,1.0,0.0,0.0,0.0,6.0,1.0,19.0,19.0,0.0,0.0,0.0,19.0,36.0,496.0,521.0,2.0,b'TRUE'
2,36.0,1.0,1.0,100.0,1.0,1.0,10.0,2.0,32.0,31.0,0.0,0.0,3.0,31.0,65.0,878.0,1150.0,3.0,b'TRUE'
3,5.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,3.0,3.0,0.0,0.0,0.0,3.0,9.0,17.0,27.0,2.0,b'TRUE'
4,55.0,3.0,5.0,0.0,3.0,2.0,7.0,0.0,19.0,4.0,0.0,0.0,0.0,4.0,5.0,30.0,42.0,2.0,b'TRUE'


In [15]:
x = df.drop(['isDefective'],axis = 'columns')
x.head()

,LOC,AverageComplexity,MaximumComplexity,LCOM,DIT,IFANIN,CBO,NOC,RFC,NIM,NIV,NCM,NCV,WMC,change_times,plus,minus,defect_num
0,588.0,2.0,7.0,97.0,2.0,1.0,54.0,0.0,65.0,20.0,11.0,23.0,23.0,43.0,166.0,2210.0,3179.0,8.0
1,21.0,1.0,1.0,0.0,0.0,0.0,6.0,1.0,19.0,19.0,0.0,0.0,0.0,19.0,36.0,496.0,521.0,2.0
2,36.0,1.0,1.0,100.0,1.0,1.0,10.0,2.0,32.0,31.0,0.0,0.0,3.0,31.0,65.0,878.0,1150.0,3.0
3,5.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,3.0,3.0,0.0,0.0,0.0,3.0,9.0,17.0,27.0,2.0
4,55.0,3.0,5.0,0.0,3.0,2.0,7.0,0.0,19.0,4.0,0.0,0.0,0.0,4.0,5.0,30.0,42.0,2.0


In [18]:
x.shape

(1065, 18)

In [19]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
scalar.fit(x)
x = scalar.transform(x)

In [20]:
x = pd.DataFrame(x)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.242863,0.090909,0.159091,0.97,0.222222,0.111111,0.461538,0.000000,0.128968,0.165289,0.23913,0.589744,0.123656,0.355372,0.514019,0.239074,0.367472,1.000
1,0.008275,0.045455,0.022727,0.00,0.000000,0.000000,0.051282,0.030303,0.037698,0.157025,0.00000,0.000000,0.000000,0.157025,0.109034,0.053656,0.060224,0.250
2,0.014481,0.045455,0.022727,1.00,0.111111,0.111111,0.085470,0.060606,0.063492,0.256198,0.00000,0.000000,0.016129,0.256198,0.199377,0.094981,0.132933,0.375
3,0.001655,0.045455,0.022727,0.00,0.000000,0.000000,0.008547,0.121212,0.005952,0.024793,0.00000,0.000000,0.000000,0.024793,0.024922,0.001839,0.003121,0.250
4,0.022342,0.136364,0.113636,0.00,0.333333,0.222222,0.059829,0.000000,0.037698,0.033058,0.00000,0.000000,0.000000,0.033058,0.012461,0.003245,0.004855,0.250


In [21]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters = 2)
kmeans_model.fit(x)

kmeans_prediction = kmeans_model.predict(x)
x['cluster'] = kmeans_prediction

kmeans_prediction[:5]

array([1, 0, 1, 0, 0], dtype=int32)

In [22]:
def ASFM(DataFrame):
    SFM = 0
    
    x,y = DataFrame.shape
    
    for i in range(x):
        for j in range(y-1):
            SFM += DataFrame.iloc[i,j]
    
    return SFM/x

In [23]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]

In [24]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.7746001183272463
ASFM1 : 2.2651560293907034


In [26]:
#ASFM1 value is greater than ASFM0, so label 1 will represent defected software and 0 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'TRUE'
    clean = b'FALSE'
    
    if defect == buggy and kmeans_prediction[i]==1:
        TP += 1
    elif defect == buggy and kmeans_prediction[i]==0:
        FN += 1
    elif defect == clean and kmeans_prediction[i]==0:
        TN += 1
    else:
        FP += 1
 

In [27]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.19458358662661607
Accuracy : 0.6394366197183099
precison : 0.35279805352798055
Recall : 0.5513307984790875
F1-Score : 0.4302670623145401


In [28]:
from sklearn.cluster import AgglomerativeClustering

ag_model = AgglomerativeClustering(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
ag_model.fit(x)
ag_prediction = ag_model.labels_
x['cluster'] = ag_prediction

ag_prediction[:5]

array([0, 1, 0, 1, 1])

In [29]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [30]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.0783028745446144
ASFM1 : 0.7172064606421298


In [31]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'TRUE'
    clean = b'FALSE'
    
    if defect == buggy and ag_prediction[i]==0:
        TP += 1
    elif defect == buggy and ag_prediction[i]==1:
        FN += 1
    elif defect == clean and ag_prediction[i]==1:
        TN += 1
    else:
        FP += 1
    

In [32]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.16484773614397677
Accuracy : 0.5887323943661972
precison : 0.32323232323232326
Recall : 0.6083650190114068
F1-Score : 0.4221635883905014


In [42]:
from sklearn.mixture import GaussianMixture

gm_model = GaussianMixture(n_components = 2)

x = x.drop(['cluster'],axis = 'columns')
gm_model.fit(x)
gm_prediction = gm_model.predict(x)
x['cluster'] = gm_prediction

gm_prediction[:5]

array([1, 1, 1, 0, 0])

In [43]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [44]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.9611253729082898
ASFM1 : 2.5411600420619593


In [47]:
#ASFM1 value is greater than ASFM0, so label 1 will represent defected software and 0 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and gm_prediction[i]==0:
        TP += 1
    elif defect == buggy and gm_prediction[i]==1:
        FN += 1
    elif defect == clean and gm_prediction[i]==1:
        TN += 1
    else:
        FN += 1
  

In [49]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.436747201041129
Accuracy : 0.7211267605633803
precison : 1.0
Recall : 0.6876971608832808
F1-Score : 0.8149532710280374


In [50]:
from sklearn.cluster import SpectralClustering

SC_model = SpectralClustering(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
SC_model.fit(x)

SC_prediction = SC_model.labels_
x['cluster'] = SC_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [51]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [52]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.304298588023053
ASFM1 : 0.7916343160305618


In [55]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and SC_prediction[i]==1:
        TP += 1
    elif defect == buggy and SC_prediction[i]==0:
        FN += 1
    elif defect == clean and SC_prediction[i]==0:
        TN += 1
    else:
        FP += 1
  

In [56]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.1938026382846342
Accuracy : 0.6469483568075117
precison : 0.8169642857142857
Recall : 0.6845386533665836
F1-Score : 0.7449118046132971


In [57]:
from sklearn.cluster import MiniBatchKMeans

miniBatchKmeans_model = MiniBatchKMeans(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
miniBatchKmeans_model.fit(x)
miniBatchKmeans_prediction = miniBatchKmeans_model.labels_
x['cluster'] = miniBatchKmeans_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [58]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [59]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.2651560293907034
ASFM1 : 0.7746001183272463


In [64]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and miniBatchKmeans_prediction[i]==1:
        TP += 1
    elif defect == buggy and miniBatchKmeans_prediction[i]==0:
        FN += 1
    elif defect == clean and miniBatchKmeans_prediction[i]==0:
        TN += 1
    else:
        FP += 1
  

In [65]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.19458358662661607
Accuracy : 0.6394366197183099
precison : 0.8195718654434251
Recall : 0.6683291770573566
F1-Score : 0.7362637362637363


In [66]:
from sklearn.cluster import Birch

birch_model = Birch(threshold=0.01, n_clusters=2)

x = x.drop(['cluster'],axis = 'columns')
birch_model.fit(x)
birch_prediction = birch_model.labels_
x['cluster'] = birch_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [67]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [68]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.8816631817845566
ASFM1 : 2.5492093920355217


In [69]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and birch_prediction[i]==0:
        TP += 1
    elif defect == buggy and birch_prediction[i]==1:
        FN += 1
    elif defect == clean and birch_prediction[i]==1:
        TN += 1
    else:
        FP += 1
  

In [70]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.24790531039079333
Accuracy : 0.7070422535211267
precison : 0.8198433420365535
Recall : 0.7830423940149626
F1-Score : 0.8010204081632653
